In [1]:
from time import time

from  sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [2]:
print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
num_data = 2000
data_samples = dataset.data[:num_data]
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 0.827s.


In [3]:
data_samples[0]

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [5]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=50000,
                                stop_words='english',
                                analyzer="word")
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

done in 0.265s.


In [10]:
import numpy as np

np.max(tf.toarray()[0])

4

In [23]:

lda = LatentDirichletAllocation(n_topics=10, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

done in 2.021s.


In [33]:
lda.transform(tf[0]).shape

(1, 10)

In [34]:
lda.get_params()

{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 50.0,
 'max_doc_update_iter': 100,
 'max_iter': 5,
 'mean_change_tol': 0.001,
 'n_jobs': 1,
 'n_topics': 10,
 'perp_tol': 0.1,
 'random_state': 0,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}

In [62]:
from  sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn.linear_model import LinearRegression

class TopicModeling(LatentDirichletAllocation):
    def __init__(self, num_topics, num_latent=None, *args, **kwargs):
        if num_latent is None:
            num_latent = num_topics
        self.num_topics = num_topics
        super().__init__(n_topics=num_latent, *args, **kwargs)
        self.label_model = LinearRegression(fit_intercept=False)
        self.vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=50000,
                                stop_words='english',
                                analyzer="word")
        
    def fit_label(self, X, y):
        X = self.transform(X)
        y = self._get_onehot(y)
        self.label_model.fit(X, y)
        
    def fit(self, X, *args, **kwargs):
        X = self.vectorizer.fit_transform(X, *args, **kwargs)
        super().fit(X)
        
    def transform(self, X, *args, **kwargs):
        X = self.vectorizer.transform(X)
        return super().transform(X, *args, **kwargs)
    
    def fit_transform(self, X, *args, **kwargs):
        self.fit(X, *args, **kwargs)
        return self.transform(X, *args, **kwargs)
        
    def predict_topic(self, X):
        X = self.transform(X)
        y = self.label_model.predict(X)
        return (y.T / np.sum(y, axis=1)).T
        
    def _get_onehot(self, y):
        return np.eye(self.num_topics)[y]

In [63]:
model = TopicModeling(10)
model.fit(data_samples)
model.fit_label(data_samples[:10], np.arange(10))

/home/tomoaki/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [66]:
model.predict_topic(data_samples)

array([[  1.00000000e+00,  -1.62897468e-11,  -7.46222628e-11, ...,
         -3.58103304e-13,  -2.07112051e-13,  -3.52361201e-11],
       [ -7.35034256e-12,   1.00000000e+00,  -6.39864273e-11, ...,
         -8.97837360e-13,  -6.31023012e-14,   2.74937711e-11],
       [  3.60676272e-11,   1.28103111e-12,   1.00000000e+00, ...,
         -3.65146021e-12,   9.45245076e-13,  -9.62640993e-11],
       ..., 
       [  8.97256173e+05,  -2.13116826e+05,  -1.24604573e+06, ...,
         -1.43554633e+05,   1.61083721e+04,  -7.08308609e+05],
       [ -2.87794768e-01,  -9.84626770e-02,   5.14627146e-01, ...,
          1.22039714e+00,  -6.67368569e-02,   5.83717091e-01],
       [  1.45443262e+00,   7.76819021e-01,   1.10297278e-01, ...,
          5.23250733e-01,  -5.87202587e-02,  -5.06815838e+00]])